In [433]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np
from statistics import mode
import re
from pycaret.classification import *
from keras.models import Sequential
from keras.layers import Dense

# Read the data
X = pd.read_csv('data/train.csv')
X_test_full = pd.read_csv('data/test.csv')

In [434]:
full = pd.concat([X, X_test_full])
full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [435]:
# mode is the value which appears the most often, it's S for embarked
# mode(full["Embarked"]) # => 'S'
full["Embarked"] = full["Embarked"].fillna(mode(full["Embarked"]))

In [436]:
 # fill missing cabins with new U type
full['Cabin'] = full['Cabin'].fillna('U')

In [437]:
# the first letters of the cabins seem to be referring to a specific part of the boat
# keep only the initials in the Cabin column
full['Cabin'] = full['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())

In [438]:
# in age column Pclass has the highest correlation, so we use that to fill missing values
full['Age'] = full.groupby("Pclass")['Age'].transform(lambda x: x.fillna(x.median()))

In [439]:
# in Fare column Pclass has the highest correlation, so we use that to fill missing values
full['Fare'] = full.groupby("Pclass")['Fare'].transform(lambda x: x.fillna(x.median()))

In [440]:
# get a unique list of Initials from Cabin types
full['Cabin'].unique().tolist()

['U', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T']

In [441]:
# Extract the salutation! A few letters with a . in the end
full['Title'] = full.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
full['Title'].unique().tolist()

['Mr',
 'Mrs',
 'Miss',
 'Master',
 'Don',
 'Rev',
 'Dr',
 'Mme',
 'Ms',
 'Major',
 'Lady',
 'Sir',
 'Mlle',
 'Col',
 'Capt',
 'Countess',
 'Jonkheer',
 'Dona']

In [442]:
# Sibsp is the number of siblings / spouses aboard
# Parch is the number of parents / children aboard
# So we calculate the family size
full['familySize'] = full['SibSp'] + full['Parch'] + 1

In [443]:
# Drop redundant features
full = full.drop(['Name', 'SibSp', 'Parch', 'Ticket'], axis = 1)

In [444]:
# One hot encoded
full_ohe = pd.get_dummies(full)



In [445]:
# recover the original test and train dataset
train = full_ohe[full_ohe['Survived'].notna()]
test = full_ohe[full_ohe['Survived'].isna()].drop(['Survived'], axis = 1)

In [446]:
# convert Survived values from float to int, it became float when concatenating
train.loc[:,'Survived'] = train.loc[:,'Survived'].astype(np.int8)

In [447]:
train.shape

(891, 38)

In [451]:
# Neural Network
nn = Sequential()
nn.add(Dense(152, kernel_initializer = 'uniform', activation = 'relu', input_dim = 37))
nn.add(Dense(304, kernel_initializer = 'uniform', activation = 'relu'))
nn.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))
nn.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [452]:
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 152)               5776      
_________________________________________________________________
dense_23 (Dense)             (None, 304)               46512     
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 305       
Total params: 52,593
Trainable params: 52,593
Non-trainable params: 0
_________________________________________________________________


In [453]:
nn.fit(train.drop(['Survived'], axis=1), train.Survived, batch_size=64, epochs=250)

Epoch 1/250
891/891 [==============================] - 1s 787us/step - loss: 0.8596 - accuracy: 0.5354
Epoch 2/250
891/891 [==============================] - 0s 74us/step - loss: 0.6222 - accuracy: 0.6622
Epoch 3/250
891/891 [==============================] - 0s 76us/step - loss: 0.6055 - accuracy: 0.6768
Epoch 4/250
891/891 [==============================] - 0s 75us/step - loss: 0.6057 - accuracy: 0.6857
Epoch 5/250
891/891 [==============================] - 0s 75us/step - loss: 0.5806 - accuracy: 0.7026
Epoch 6/250
891/891 [==============================] - 0s 76us/step - loss: 0.5935 - accuracy: 0.6869
Epoch 7/250
891/891 [==============================] - 0s 80us/step - loss: 0.5539 - accuracy: 0.7104
Epoch 8/250
891/891 [==============================] - 0s 76us/step - loss: 0.5553 - accuracy: 0.7228
Epoch 9/250
891/891 [==============================] - 0s 74us/step - loss: 0.5772 - accuracy: 0.7205
Epoch 10/250
891/891 [==============================] - 0s 75us/step - loss: 0.54

891/891 [==============================] - 0s 77us/step - loss: 0.3759 - accuracy: 0.8485
Epoch 81/250
891/891 [==============================] - 0s 75us/step - loss: 0.3699 - accuracy: 0.8418
Epoch 82/250
891/891 [==============================] - 0s 75us/step - loss: 0.3720 - accuracy: 0.8406
Epoch 83/250
891/891 [==============================] - 0s 73us/step - loss: 0.3647 - accuracy: 0.8496
Epoch 84/250
891/891 [==============================] - 0s 76us/step - loss: 0.3781 - accuracy: 0.8305
Epoch 85/250
891/891 [==============================] - 0s 78us/step - loss: 0.3548 - accuracy: 0.8507
Epoch 86/250
891/891 [==============================] - 0s 84us/step - loss: 0.3745 - accuracy: 0.8328
Epoch 87/250
891/891 [==============================] - 0s 90us/step - loss: 0.3952 - accuracy: 0.8227
Epoch 88/250
891/891 [==============================] - 0s 83us/step - loss: 0.3642 - accuracy: 0.8485
Epoch 89/250
891/891 [==============================] - 0s 82us/step - loss: 0.3543 - 

891/891 [==============================] - 0s 91us/step - loss: 0.3296 - accuracy: 0.8530
Epoch 160/250
891/891 [==============================] - 0s 89us/step - loss: 0.3196 - accuracy: 0.8653
Epoch 161/250
891/891 [==============================] - 0s 90us/step - loss: 0.3190 - accuracy: 0.8631
Epoch 162/250
891/891 [==============================] - 0s 94us/step - loss: 0.3128 - accuracy: 0.8642
Epoch 163/250
891/891 [==============================] - 0s 92us/step - loss: 0.3142 - accuracy: 0.8653
Epoch 164/250
891/891 [==============================] - 0s 92us/step - loss: 0.3216 - accuracy: 0.8676
Epoch 165/250
891/891 [==============================] - 0s 89us/step - loss: 0.3178 - accuracy: 0.8608
Epoch 166/250
891/891 [==============================] - 0s 88us/step - loss: 0.3199 - accuracy: 0.8777
Epoch 167/250
891/891 [==============================] - 0s 90us/step - loss: 0.3403 - accuracy: 0.8676
Epoch 168/250
891/891 [==============================] - 0s 88us/step - loss: 

891/891 [==============================] - 0s 99us/step - loss: 0.2864 - accuracy: 0.8732
Epoch 238/250
891/891 [==============================] - 0s 102us/step - loss: 0.2840 - accuracy: 0.8721
Epoch 239/250
891/891 [==============================] - 0s 96us/step - loss: 0.3098 - accuracy: 0.8586
Epoch 240/250
891/891 [==============================] - 0s 93us/step - loss: 0.2971 - accuracy: 0.8709
Epoch 241/250
891/891 [==============================] - 0s 95us/step - loss: 0.2875 - accuracy: 0.8754
Epoch 242/250
891/891 [==============================] - 0s 95us/step - loss: 0.2685 - accuracy: 0.8810
Epoch 243/250
891/891 [==============================] - 0s 96us/step - loss: 0.2827 - accuracy: 0.8822
Epoch 244/250
891/891 [==============================] - 0s 95us/step - loss: 0.2944 - accuracy: 0.8765
Epoch 245/250
891/891 [==============================] - 0s 92us/step - loss: 0.3066 - accuracy: 0.8788
Epoch 246/250
891/891 [==============================] - 0s 93us/step - loss:

In [454]:
score, acc = nn.evaluate(train.drop(['Survived'], axis=1), train.Survived, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

891/891 [==============================] - 0s 150us/step
Test score: 0.2889716106265215
Test accuracy: 0.872053861618042


In [455]:
# Make predictions on test set
test['Survived'] = nn.predict_classes(test)

In [456]:
test

,PassengerId,Pclass,Age,Fare,familySize,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Survived
0,892,3,34.5,7.8292,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,893,3,47.0,7.0000,2,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,894,2,62.0,9.6875,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,895,3,27.0,8.6625,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,896,3,22.0,12.2875,3,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,24.0,8.0500,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
414,1306,1,39.0,108.9000,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
415,1307,3,38.5,7.2500,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
416,1308,3,24.0,8.0500,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [457]:
# Create file for submission
test[['PassengerId', 'Survived']].to_csv('data/submission_FE_nn.csv', index = False)